In [ ]:
import os
import train_model
from aws_helpers import create_repository, setup_elb, create_ecs_service, deregister_old_taskdefinitions
from keras_to_tfserving import convert_keras_to_tf_model

### Reqs.
This code requires a couple of things:
- Installed requirements
- Docker (https://docs.docker.com/install/linux/docker-ce/ubuntu/)
- installed & configured AWS CLI (https://aws.amazon.com/cli/)
- AWS Elastic Load Balancer setup as an Application Load Balancer with a listener on 443
- AWS Elastic Container Srevice cluster setup and configured

In [ ]:
version = 1
service_path = 'services.dev.sellpy.net'
service = "stockholm-ai"
cluster = "microservices"
env = "dev"
load_balancer_name = "sellpy-services"

#### Train model & Convert model

In [ ]:
model_name = "stockholm_ai_mnist.m"
!ls
if model_name not in os.listdir("."):
    train_model.output_model(model_name, epochs=1)

# note that the model path is referred to in server.conf
# TODO: Make this repeatable in notebook
convert_keras_to_tf_model(model_name=model_name,
                          model_path="models/stockholm-ai",
                          sequential=True)

#### Create a repository on ECR, build & push the image

In [ ]:
# Create an image repository for storing docker files.
create_repository(service)

!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev


#### Register a target & listener in the load balancer

In [ ]:
target_group_arn = setup_elb(
    service=service,
    version=version,
    load_balancer_name=load_balancer_name,
    service_path=service_path
)

#### Register a task definition and create an ECS service

In [ ]:
# Create / Update a task definition.
!aws ecs register-task-definition --cli-input-json file://task_definition_2.json --region eu-west-1
create_ecs_service(
    cluster=cluster,
    service=service,
    task_definition_family=service,
    target_group_arn=target_group_arn,
    service_count=1
)

In [ ]:
# Update existing services
!aws ecs register-task-definition --cli-input-json file://task_definition_2.json --region eu-west-1
!aws ecs update-service --service stockholm-ai --task-definition stockholm-ai --region eu-west-1  --cluster microservices
deregister_old_taskdefinitions(service)